In [14]:
import napari
from napari import layers
from typing import NewType, List, Tuple, Any, Optional, Type, Union
import numpy as np

import magicgui
from magicgui import register_type
from magicgui.widgets._bases import CategoricalWidget
from magicgui.widgets import FunctionGui

from napari.utils._magicgui import find_viewer_ancestor

In [2]:
curvature = NewType("mean_curvature", np.ndarray)
curvature

<function typing.NewType.<locals>.new_type(x)>

In [3]:
def get_layers_features(gui: CategoricalWidget) -> List[layers.Layer]:
    """Retrieve layers matching gui.annotation, from the Viewer the gui is in.
    Parameters
    ----------
    gui : magicgui.widgets.Widget
        The instantiated MagicGui widget.  May or may not be docked in a
        dock widget.
    Returns
    -------
    tuple
        Tuple of layers of type ``gui.annotation``
    Examples
    --------
    This allows the user to do this, and get a dropdown box in their GUI
    that shows the available image layers.
    >>> @magicgui
    ... def get_layer_mean(layer: napari.layers.Image) -> float:
    ...     return layer.data.mean()
    """
    if not (viewer := find_viewer_ancestor(gui.native)):
        return ()

    search_key = gui.annotation.__name__
    search_fields = list(layer.features.keys()) + list(layer.metadata.keys())
    
    return [layer for layer in viewer.layers if search_key in search_fields]

In [4]:
register_type(
    curvature,
    choices = get_layers_features
)

<function typing.NewType.<locals>.new_type(x)>

In [5]:
def test_function(mean_curvature: curvature) -> curvature:
    
    layer = None
    if isinstance(mean_curvature, layers.Layer):
        layer = mean_curvature
        mean_curvature = np.asarray(mean_curvature.features[curvature.__name__])
        
    result = mean_curvature * 10
    if layer is not None:
        layer.features['new_result'] = result
    return result

In [6]:
viewer = napari.Viewer()

Assistant skips harvesting pyclesperanto as it's not installed.


In [7]:
viewer.add_points(np.random.random((100,3)), features = {'mean_curvature': np.ones(100)})

<Points layer 'Points' at 0x22c7336a850>

In [8]:
widget = magicgui.magicgui(test_function)
widget = viewer.window.add_dock_widget(widget)

In [10]:
widget()
viewer.layers[-1].features

,mean_curvature,new_result
0,1.0,10.0
1,1.0,10.0
2,1.0,10.0
3,1.0,10.0
4,1.0,10.0
...,...,...
95,1.0,10.0
96,1.0,10.0
97,1.0,10.0
98,1.0,10.0


## different approach: union types

In [20]:
def test_function2(arg1: Union[layers.Points, np.ndarray]) -> np.ndarray:
    return arg1
                               

In [23]:
magicgui.__version__

'0.5.1'